<p align="left">[Voltar à tabela de conteúdos](README.md#conteúdo)<p>

# Iterabilidade

Esse tópico estuda como o Python implementa a ideia de iterabilidade. Abordamos primeiramente as classes gerais da iteração: iteráveis e iteradores. Fazemos alguns exemplos e depois abordamos os iteradores especiais, i.e., iteradores de sequências e geradores. Por ser mais complexo e auxiliar, este último tipo pode ser pulado em uma primeira leitura. Por fim, na seção bônus, apresentamos algumas instâncias onde a iteração aparece naturalmente na linguagem Python: compreensão de tuplas, o loop ```for```, o operador ```in```, e a ideia de *lazy evaluation*.



## Objetos fundamentais da iteração

De início, vale lembrar que tudo em Python é um objeto. Há duas categorias de objetos em Python que permitem a implementação da iterabilidade:
* objetos iteráveis (iterables); e
* objetos iteradores (iterators).

### Iteráveis

Um objeto **iterável** é aquele capaz de retornar seus elementos um de cada vez. É iterável qualquer objeto cuja classe defina:
* ou um método de nome ```__getitem__(self, keys)``` que implemente a semântica de sequências; ou
* ou um método de nome ```__iter__(self)```.

A semântica de sequências é simplesmente a definição de um objeto cujo método ```__getitem__(self, keys)``` aceite como chaves números inteiros de 0 até levantar a exceção IndexError. Existem outras possibilidades de aceitar chaves, como os dicionários, que aceitam também strings ou hashables em geral.

Historicamente, o Python usava apenas o método ```__getitem__()``` para a iteração. Porém, a partir de Python 2.2, passou-se a usar preferencialmente o método ```__iter__()```, de modo que o uso de ```__getitem__()``` para fins de iteração passou a ser considerado legado.

O método ```__getitem__()``` apenas retorna o elemento selecionado. Já o método ```__iter__()``` retorna outro objeto, um iterador já construído. Veremos o que são iteradores abaixo.

### Iteradores

Toda vez que o Python espera um objeto iterável, ele chama a função nativa ```iter(objeto)```. Se o objeto tem um método ```__iter__(self)```, o Python chama este método. Caso contrário, o Python procura por ```__getitem__()```. Se não houver nenhum desses dois métodos, o Python levanta um erro de tipo (objeto não é iterável). Exemplos de instâncias onde o Python espera um iterável são no loop ```for``` ou no argumento da função ```list()```:

In [1]:
class Void:
  def __init__(self):
    pass

for i in Void():
  print('Deu certo.')

TypeError: 'Void' object is not iterable

In [2]:
list(Void())

TypeError: 'Void' object is not iterable

Neste ponto devemos nos perguntar: o que é retornado por esta chamada à função ```iter()```? É aqui que entra o outro tipo de objeto comentado acima, chamado **iterador**, que representa um fluxo de dados.

Quando o método ```__iter__()``` existe, ele já retorna um iterador construído. Quando existe apenas o método ```__getitem__()```, a função ```iter()``` o usa para construir um iterador especial do tipo iterador de sequência, que veremos mais adiante.

Iteradores são objetos que implementam dois protocolos:
* um método ```__iter__()``` que apenas retorna o próprio iterador, permitindo que tanto iteráveis ou iteradores sejam passados quando o Python espera um iterável; e
* um método ```__next__()``` que retorna o próximo item do iterável (notar: do iterável e não do iterador) e avança o estado da iteração.

Se precisarmos fazer várias iterações, é necessário instanciar várias vezes o iterador. Isso porque, quando o iterador termina a contagem, ele é esgotado e não pode mais ser usado. A partir de então, uma nova chamada a ```next()``` retorna a exceção ```StopIteration```:

In [3]:
iteravel = [1,2,3]
iterador = iter(iteravel)
print(next(iterador))
print(next(iterador))
print(next(iterador))
print(next(iterador))

1
2
3


StopIteration: 

### Iteráveis vs. Iteradores

Mas por que é feita uma separação entre dois tipos, iteráveis e iteradores?

Esta separação permite uma série de eficiências e flexibilidades.

Primeiro, o iterável não precisa armazenar internamente o estado da iteração, e não é necessário acessá-lo diretamente ou mudar valores de seus atributos ao iterar sobre ele. É o iterador que guarda a informação sobre o ponto em que a iteração se encontra.

Segundo, isso implica que toda vez que o Python espera um iterável, o comando em questão não precisa saber como acessar e alterar o estado do iterável, funcionando com qualquer objeto iterável em geral por meio do protocolo de iteração acima Por exemplo, um loop ```for``` ou um construtor ```list()``` não precisa saber nenhum detalhe sobre o iterável que lhes é passado.

Terceiro, há uma eficiência de memória envolvida. No caso do iterador, cada elemento só é gerado sob demanda (i.e., chamando ```next()```), o que implica que não é preciso criar, modificar ou duplicar tudo o que vai ser iterado na memória. Um exemplo clássico é o iterável range, que nunca cria toda a sequência na memória. Ele apenas armazena uma tupla ```(start, stop, step)```, gerando o próximo elemento sob demanda, e seu iterador estoca o estado interno e solicita o próximo estado quando necessário:

In [4]:
r = range(10**8) # sequência gigante
iterador = iter(r)
next(iterador); next(iterador)

1

Quarto, por fim, podemos ter várias iterações diferentes sobre um único iterável. É o que acontece quando chamamos vários loops ```for``` sobre um mesmo iterável:

In [5]:
r = range(3)
for i in r:
  print(i)
for i in r:
  print(i)

0
1
2
0
1
2


É também o que acontece se realizamos novas iterações mesmo que as anteriores ainda não tenham sido finalizadas. Por exemplo, se temos uma corrida entre dois jogadores sobre um único circuito, podemos criar dois iteradores diferentes:

In [6]:
# supomos que a casa inicial é a de número 0
circuito = (1,2,3,4,5)
jogador_A = iter(circuito)
jogador_B = iter(circuito)

# anda uma casa com o jogador A e mostra seu estado atual
print(f'Jogador A: casa {next(jogador_A)}')

# anda duas casas com o jogador B
next(jogador_B); next(jogador_B)

# anda mais uma casa com o jogador B e mostra seu estado atual
print(f'Jogador B: casa {next(jogador_B)}')


Jogador A: casa 1
Jogador B: casa 3


Vale observar que o método ```__iter()__``` retorna o próprio iterador, e não uma nova instância do iterador. Assim, se o iterador em questão estiver esgotado, chamar sobre ele ```iter()``` apenas retorna o mesmo iterador esgotado. Do mesmo modo, se ele estiver em algum estágio intermediário da iteração, ```iter()``` não retorna um iterador com a contagem resetada.

## Exemplos práticos

### Método antigo: semântica de sequência

Abaixo criamos uma classe de iteráveis chamada ```Reverso``` pelo método ```__getitem__()```. Esta classe apenas reverte a indexação de uma lista ou sequência:

In [7]:
# sequência que reverte a ordem original
class Reverso:
  def __init__(self, sequencia):
    if not hasattr(sequencia, '__getitem__'):
      raise TypeError('Argumento sequencia deve ser do tipo Sequence.')
    self.sequencia_original = sequencia
  def __getitem__(self, keys):
    if type(keys) is not int or keys < 0:
      raise TypeError('Objeto da classe Reverso deve ser indexado por um int não negativo.')
    if keys >= len(self.sequencia_original):
      raise IndexError('Índice fora de alcance.')
    return self.sequencia_original[-(keys+1)]

# teste
for r in Reverso([*'abcd']):
  print(r)

d
c
b
a


### Dupla iterável-iterador

Agora criaremos uma classe de iteráveis pelo método ```__iter__()```, junto com uma classe de iteradores implementando o protocolo de métodos ```__iter__()``` e ```__next__()```. O iterável é uma caminhada aleatória clássica em duas dimensões e um número finito de passos:

In [8]:
import random

# caminhada aleatória 2D clássica
class Caminhada:
  def __init__(self, estado_inicial=0, passos_totais=10):
    self.estado = estado_inicial
    self.passos_totais = passos_totais
  def __iter__(self):
    return Caminhada.Passo(self)

  class Passo:
    def __init__(self, caminhada):
      self.caminhada = caminhada
      self.passo = 0
    def __iter__(self):
      return self
    def __next__(self):
      if self.passo >= self.caminhada.passos_totais:
          raise StopIteration
      self.caminhada.estado += random.choice([-1,1])
      self.passo += 1
      return self.caminhada.estado

# teste
c = Caminhada()
caminho = f'{c.estado}'
for p in c:
  caminho += f'  =>  {p}'

print(f'Caminho realizado:\n{caminho}')

Caminho realizado:
0  =>  -1  =>  0  =>  1  =>  2  =>  3  =>  2  =>  3  =>  4  =>  5  =>  4


Notar que ambos os exemplos funcionam com o loop ```for```. Notar, ainda, que seria perfeitamente aceitável definir a classe de iteradores fora da classe de iteráveis. Essa é uma escolha que acarreta vantagens e desvantagens em termos de legibilidade, testabilidade e escopo.

### Somente iterador

Um jeito alternativo de definir a caminhada aleatória deriva da constatação de que este iterável é tão simples que sequer estoca internamente o caminho percorrido, de forma que ele poderia ser apenas um iterador (sem iterável). Tendo em vista que o iterador também possui o método ```__iter__()```, ele também retorna o que o Python necessita quando se espera por um iterável.

Precisamos somente ajustar o que é retornado e quando a iteração termina, pois agora queremos exibir o resultado da primeira iteração como sendo o estado inicial. Implementamos essa simplificação abaixo:

In [9]:
# simplificação da caminhada aleatória
class Caminhada_iterador:
  def __init__(self, estado_inicial=0, passos_totais=10):
    self.estado = estado_inicial
    self.passos_totais = passos_totais
    self.passo = 0
  def __iter__(self):
    return self
  def __next__(self):
    if self.passo > self.passos_totais:
        raise StopIteration
    atual = self.estado # guarda o passo atual
    self.estado += random.choice([-1,1]) # atualiza para a próxima chamada
    self.passo += 1
    return atual

# teste
cIt = Caminhada()
caminho = f'{cIt.estado}'
for p in cIt:
  caminho += f'  =>  {p}'

print(f'Caminho realizado:\n{caminho}')

Caminho realizado:
0  =>  -1  =>  -2  =>  -1  =>  -2  =>  -1  =>  -2  =>  -1  =>  -2  =>  -1  =>  0


Ao definirmos apenas iteradores sem iteráveis, temos de tomar o cuidado de sempre instanciar o iterador a cada nova iteração, visto que o iterador se esgota a cada iteração completa. No código acima isso não foi necessário, pois iteramos apenas uma vez, mas veremos mais à frente casos em que isso será necessário.



## Iteradores especiais

### Iteradores de sequências

No caso de sequências, quando só existe o método ```__getitem__()```, o objeto não fornece um método para construir um iterador. Em vez disso, o Python cria automaticamente um objeto do tipo *iterador de sequência*, que começa com uma variável de índice inicializada em ```0``` (digamos, ```i```) e, quando lhe é aplicada a função ```next()```, chama ```__getitem__(i)``` e incrementa ```i```. Se ```__getitem__(i)``` levantar a exceção ```IndexError```, entende-se que a sequência acabou, sendo levantada a exceção ```StopIteration```.

Vale notar que o Python não tem como saber que a indexação implementada naquele objeto segue à risca a semântica de sequências. Desta forma, esse sequenciamento pode ser enganado, como nos dois exemplos abaixo:

In [10]:
# oculta o primeiro elemento de uma lista
class Lista_oculta:
  def __init__(self, lista):
    if type(lista) is not list:
      raise TypeError('Tipo deve ser list.')
    self.lista_original = lista

  def __getitem__(self, keys):
    return self.lista_original[keys+1]

# teste
for i in Lista_oculta([1,2,3,4]):
  print(i)

2
3
4


In [11]:
import numpy as np

# só revela marcações inteiras
class Pontos_ocultos:
  def __init__(self, fim, espacamento=0.25):
    start = 0
    self.marcacoes = np.arange(start, fim, espacamento)

  def __getitem__(self, keys):
    try:
      indice = list(self.marcacoes).index(keys)
      return self.marcacoes[indice]
    except ValueError: # o método .index() levanta essa exceção quando o valor não existe na lista
      raise IndexError('Indice fora de alcance.')

# teste da indexação float
po = Pontos_ocultos(5)
indice_float = po[0.25]
print(f'Indexação float:\n{indice_float}')

# teste da interação de marcações inteiras
print('\nLoop:')
for i in po:
  print(i)

Indexação float:
0.25

Loop:
0.0
1.0
2.0
3.0
4.0


Neste último exemplo, notar que foi necessário levantar uma exceção manualmente porque, caso contrário, uma exceção de ```ValueError``` seria levantada antes da exceção de ```IndexError```, o que faria com que o Python nunca parasse de procurar pelo próximo item!

(Obs.: A exceção ```StopIteration``` também poderia ter sido levantada no código acima, produzindo o mesmo efeito de ```IndexError```, porém esta última é mais recomendada por ser mais geral, visto que funcionaria corretamente caso o usuário tentasse manualmente acessar um elemento fora de alcance.)

### Geradores

O conceito de geradores foi introduzido oficialmente somente em Python 2.3, depois do protocolo de iteração explicado acima (que remonta a Python 2.2). Sua ideia geral é iterar dentro uma função, permitindo que a função seja apenas parcialmente executada, pausada, e depois retomada de onde parou, até mesmo sucessivas vezes se necessário.

Há três conceitos envolvidos na implementação de geradores em Python:
* **iterador gerador**: um tipo especial de iterador;
* **função geradora**: uma função que retorna uma instância de iterador gerador; e
* **expressão geradora**: uma expressão que retorna uma instância de iterador gerador.

Comecemos introduzindo o conceito de função geradora, depois retomamos os demais.

#### Função geradora

Para transformar uma função comum em uma função geradora, basta introduzir a instrução ```yield``` no corpo da função. A função retorna um objeto iterador do tipo gerador, já devidamente construído - ou seja, é possível invocar ```iter()``` ou ```next()``` sobre o objeto retornado.

Na primeira iteração, executa-se o corpo da função até atingir a instrução ```yield```, retornando o que esta intrução indicar (se nada for indicado, retorna ```None```). O ponto onde paramos a execução (estado de execução) fica estocado no iterador gerador. Ao chamarmos uma nova iteração, a execução da função é continuada até o próximo comando ```yield```, e assim sucessivamente. Caso não haja mais nenhum comando desta natureza, o iterador gera uma exceção ```StopIteration```.

O exemplo simples abaixo ilustra esse funcionamento. Podemos notar que o parênteses no valor depois do comando ```yield``` é opcional. Vemos também que é possível a função incluir um ```return```, que será passado à exceção ```StopIteration```. Caso isso não seja incluído, passa-se apenas ```None```.

In [12]:
def gen():
  yield 'a'
  yield 'b'
  yield ('c')
  return 'Fim.'

it_gen = gen()
print(next(it_gen))
print(next(it_gen))
print(next(it_gen))
print(next(it_gen))

a
b
c


StopIteration: Fim.

#### Objeto iterador gerador

Além dos métodos ```__iter()___``` e ```__next__()```, o iterador gerador também vem com outros três métodos já embutidos. Todos esses métodos continuam a execução da função, à semelhança de ```next()```, mas com particularidades:
* ```.send(self, valor)```: substitui o retorno da última expressão ```yield``` executada pelo valor de ```valor```, executando novamente a linha lógica onde ela se encontrava. Não pode ser usado na primeira iteração, apenas em iterações subsequentes, sob pena de levantar a exceção ```TypeError```.
* ```.throw(self, valor)```: levanta a exceção especificada em ```valor``` no ponto de execução em que o gerador se encontra. Se a exceção for capturada, executa o ```yield``` subsequente àquele que gerou a exceção. Se não houver mais nenhum ```yield```, o gerador levanta uma exceção ```StopIteration``` (mesmo que a exceção dada por ```throw()``` tenha sido capturada). Também pode ser usado na primeira iteração, apenas em iterações subsequentes, sob pena de levantar a exceção ```TypeError```. (Obs.: versões deprecadas desse método aceitam argumentos adicionais.)
* ```.close(self)```: levanta uma exceção especial chamada ```GeneratorExit```, que esgota o iterador e, se capturada pela função e for usada depois da primeira iteração, permite a execução de instrução pela cláusula ```finally```. Por padrão, o método retorna ```None```. Caso a função tente devolver algum valor por meio de um novo comando ```yield```, levanta-se um ```RunTimeError```. A partir de Python 3.3, pode-se retornar um valor final, que será passado para dentro da exceção ```StopIteration``` sob o atributo ```StopIteration.value```.

Vale notar que os métodos podem tentar capturar exceções normalmente, mas as várias exceções acima podem mesmo assim causar um erro.

Os exemplos abaixo ilustram o uso desses métodos:

In [13]:
# .send()

def g():        # 1ª iteração: devolve 1º yield ('a')
  t = yield 'a' # 2ª iteração com .send: roda novamente esta linha
                # inserindo na atribuição o valor passado a .send(valor)
  yield t

# teste com .send()
it = g()
print(next(it)) # 1ª iteração
print(it.send('b')) # 2ª iteração

# teste sem .send()
print()
it = g()
print(next(it))
print(next(it)) # como nada foi alimentado para substituir "yield 'a'", retorna None

# erro ao usar .send() na primeira iteração
it = g()
print(it.send('c'))

a
b

a
None


TypeError: can't send non-None value to a just-started generator

In [14]:
# throw

def gen():
  try:
    yield 'a'
    yield 'b'
  except:
    print('Exceção gerada.')
  yield 'c'

it = gen()
print(next(it))
print(it.throw(TypeError))

a
Exceção gerada.
c


In [15]:
# close
def gen():
  try:
    yield 'a'
    yield 'b'
  finally:
    print('Gerador encerrado.')

it = gen()
print(next(it))
it.close()
next(it)

a
Gerador encerrado.


StopIteration: 

De início, os usos de ```.send()``` podem ser confusos. Para garantir o entendimento, vejamos o exemplo abaixo.


In [16]:
def gen():
  yield 'desperdicado'
  g = yield
  yield g + ' foi guardado.'

it = gen()
print(next(it))
print(next(it))
print(it.send('valor'))

desperdicado
None
valor foi guardado.


A primeira iteração retorna uma string que não é salva em nenhum lugar. A iteração posterior retorna ```None```, pois a instrução ```yield``` não veio acompanhada de nenhum valor. Chamamos então ```.send('valor')```, que produz um código equivalente ao seguinte:

```
g = 'valor'
yield g + ' foi guardado'
```

Isso ocorre porque ```.send()``` executa as seguintes operações:

1. Substitui toda a expressão ```yield``` da iteração anterior pelo argumento que lhe foi passado

2. Executa a linha lógica onde esta substituição aconteceu

3. Continua a execução da função

O exemplo abaixo é mais complexo. O iterador é inicializado com o valor ```0```. O loop então começa. Na sua primeira execução, envia para o gerador o valor de ```x```, que é atribuído ao incremento já dentro do loop infinito na função. Este valor de incremento é usado para se somar à contagem. A função continua executando: seu loop recomeça e esbarra na próxima devolução, o valor acumulado da contagem, que é impresso pelo loop externo. E assim por diante, até o loop externo acabar.

In [17]:
# soma o indice do elemento anterior
def gen():
  cumulativo = 0
  while True:
    incremento = yield cumulativo
    cumulativo += incremento

# teste
it = gen()
print(next(it)) # iniciando para poder receber valores de send()
for x in range(5): # soma parcial dos 5 primeiros elementos
  print(it.send(x))

# encerrando
it.close()
try:
  next(it)
except StopIteration:
  print('-- Concluído --')

0
0
1
3
6
10
-- Concluído --


#### Expressão geradora

É possível criar geradores por meio de uma sintaxe semelhante à de compreensão de listas, conjuntos ou dicionários, porém entre parênteses em vez de ```[]``` ou ```{}```.

A vantagem de fazer esse tipo de compreensão em vez da compreensão tradicional é que os valores são criados sob demanda usando os métodos acima, em vez de serem todos estocados na memória.

In [18]:
it = (x for x in range(10))
print(f'Objeto iterador gerador: {it}') # apenas mostra o objeto, sem a sequência produzida
print(f'\n1ª iteração, gerada sob demanda: {next(it)}') # apenas gera o próximo elemento

l = [y for y in range(10)]
print(f'\nLista gerada por compreensão: {l}') # todos os valores são calculados de imediato

Objeto iterador gerador: <generator object <genexpr> at 0x00000170FF001180>

1ª iteração, gerada sob demanda: 0

Lista gerada por compreensão: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


Os parênteses podem ser omitidos quando se trata de uma chamada de único argumento e o Python espera um iterável:

In [19]:
sum(x for x in range(10))

45

#### Delegação e o comando ```yield from```



O comando ```yield from``` pode ser usado no lugar de ```yield``` para receber o que outra função geradora devolve com este último comando. A ideia é que geradores possam ser facilmente subdivididos ou modularizados, tal como fazemos com grandes funções.

Suponha que queremos gerar uma caminhada que termina em um momento aleatório de tempo. Não precisamos criar iteradores ou geradores do nada. Basta receber de iteradores que já existem:

In [20]:
N = 10 # limite máximo da caminhada
def fim_subito():
  yield from range(random.randint(1, N))

print(f'1ª caminhada: {list(fim_subito())}')
print(f'2ª caminhada: {list(fim_subito())}')
print(f'3ª caminhada: {list(fim_subito())}')


1ª caminhada: [0, 1, 2, 3, 4, 5]
2ª caminhada: [0, 1, 2, 3, 4, 5, 6, 7]
3ª caminhada: [0]


A vantagem desta construção é que ela permite injetar valores enviados por ```.send()``` e ```.throw()``` dentro do escopo do gerador original.

O exemplo abaixo é dado na documentação oficial do Python. Ele implementa um gerador que serve de contador, sendo incrementado pelo número recebido via ```.send()```, e reseta se receber ```None```:

In [21]:
def acumular():
    contador = 0
    while 1:
        proximo = yield # quando send é chamada, substitui yield por seu argumento,
                        # executa esta atribuição, e continua executando até
                        # encontrar yield novamente ou até encontrar return
        if proximo is None:
            return contador
        contador += proximo

def juntar_contadores(contadores):
    while 1:
        contador = yield from acumular()
        contadores.append(contador)

contadores = []
ac = juntar_contadores(contadores)
next(ac)  # retorna None, mas se assegura de que acumular() pode começar a receber valores
for i in range(4):
    ac.send(i)

ac.send(None)  # termina o primeiro contador
for i in range(5):
    ac.send(i)

ac.send(None)  # termina o segundo contador
contadores

[6, 10]

## Bônus

### A compreensão de tuplas

Como tuplas são imutáveis, não é possível criá-las usando uma compreensão direta, pois a compreensão envolve um loop que vai incrementalmente adicionando elementos a um objeto (necessariamente) mutável.

Porém, como o construtor ```tuple()``` espera um iterável, podemos fazer uma espécie de compreensão indireta usando expressões geradoras, criando uma tupla assim:

In [22]:
tup = tuple(x for x in range(3))
print(tup)

(0, 1, 2)


### Exercício: recriando o loop ```for```

Com o conhecimento adquirido acima, podemos facilmente recriar o loop for.

A sintaxe desse comando em Python exige pelo menos três elementos: um nome genérico para o elemento atual (target), o iterável que iremos percorrer, e o código que desejamos executar a cada iteração (suite). Ou seja:

```
for [target] in [iterável]:
   [suite]
```

Vamos criar uma função que recebe esses três elementos, instancia um iterador para o iterável, e o utiliza para executar o loop. A particularidade é que teremos de passar os argumentos token e suite como strings. (Por simplicidade, vamos ignorar a possibilidade de passar listas de targets usando descompressão.)


In [23]:
# função que imita o loop for
def ffor(target, iteravel, suite):
  iterador = iter(iteravel)
  while True:
    try:
      proximo_valor = next(iterador)
    except StopIteration:
      return None
    # executa o código de uma string, herdando para dentro do escopo a variável target:
    exec(suite, {}, {target: proximo_valor})

# teste
ffor('i', Reverso('abcd'), 'print(i)')

d
c
b
a


### Contâineres e o operador ```in```

Objetos do tipo contâiner são quaisquer objetos com o método ```__contains__()```, cujo conceito é estocar uma coleção de elementos. Esse método permite a operação binária ```in```, que verifica se algum dos elementos dentro do contâiner é igual ao elemento testado. Ou seja, são equivalentes:

```x in y ```

e

```y.__contains__(x)```.

E o que acontece se a classe em questão não definir tal método? O Python tenta iterar sobre o objeto usando ```__iter__()``` e ```__next__()```, e caso não consiga, usando ```__getitem__()```. Cada elemento do objeto alvo é comparado com o objeto buscado, usando tanto o operador ```in``` quanto o operador ```==``` (qualquer um que retorne ```True``` vale). Vale notar que, se tais métodos gerarem alguma exceção nesse processo, é como se o próprio operador a tivesse gerado.

Vejamos alguns exemplos.

O iterador abaixo é um contador. Notar que o iterador não estoca todos os valores da contagem, mas apenas o valor atual. Mesmo assim, o operador ```in``` localiza qualquer valor da contagem!

In [24]:
# iterador
class Contador:
  def __init__(self, N):
    self.stop = N
    self.count = 0

  def __iter__(self):
    return self

  def __next__(self):
    if self.count >= self.stop:
      raise StopIteration
    atual = self.count
    self.count += 1
    return atual

# teste
N = 5
msg = 'Contagens do contador de 5: '

for i in Contador(N): # instância 1
  msg += f' {i} ' # esgotamento da instância 1
print(msg)

valor_testado = 3
c5 = Contador(N) # instância 2
teste = valor_testado in c5
print(f'Teste se {valor_testado} está no contador: {teste}')
proximo_valor = next(c5) # instância 2 ainda não foi esgotada
print(f'Valor seguinte do iterador: {proximo_valor}')

Contagens do contador de 5:  0  1  2  3  4 
Teste se 3 está no contador: True
Valor seguinte do iterador: 4


No código acima, criamos um iterador, e não um iterável. Lembre-se que um iterador, diferentemente de um iterável, é inteiramente esgotado quando iteramos uma vez sobre ele, por isso devemos instanciar dois contadores diferentes no trecho de teste do código. A segunda instância, ```c5```, permite observar que o operador ```in``` já sai da iteração assim que recebe a primeira comparação verdadeira. Como testamos um valor intermediário no meio da contagem, tal instância do iterador ainda não está esgotada, e podemos invocar ```next()``` sobre ela sem levantar nenhuma exceção, obtendo o próximo valor da iteração.

Para terminar, criamos abaixo um objeto que não deixa uma iteração acessar elementos a partir de um certo ponto, levantando alguma exceção arbitrária. Vemos novamente que o operador ```in``` funciona, e desta vez ele levanta a mesma exceção levantada por ```__getitem__()```.

In [25]:
# sequencia com cauda oculta
class Bloqueio:
  def __init__(self, ponto_de_bloqueio, N=10):
    self.block = ponto_de_bloqueio
    self.sequencia = list(range(N))

  def __getitem__(self, keys):
    if keys >= self.block: # exceção arbitrária
      raise Exception('Tentativa de acessar valor além do bloqueio')
    return self.sequencia[keys]

# teste
s = Bloqueio(5)
print(3 in s)
print(6 in s)

True


Exception: Tentativa de acessar valor além do bloqueio

### Avaliação preguiçosa


Para iniciantes em Python, pode parecer estranho que não consigamos acessar de imediato os valores de alguns objetos. Por exemplo, quando pedimos ao Python uma sequência finita de valores, tal como em ```range()```, por que o Python simplesmente não nos mostra o que foi criado? Por que precisamos chamar a função ```list()``` para ver os valores?

In [26]:
r = range(5)
print(r)
print(list(r))

range(0, 5)
[0, 1, 2, 3, 4]


A resposta está na avaliação preguiçosa (ou *lazy evaluation*). Quando criamos um objeto da classe ```range```, os valores da sequência ainda não existem. Eles são criados sob demanda durante alguma iteração sobre este objeto. Como a função ```list()``` espera um iterável, ela invoca ```iter()```, passando por todos os procedimentos que vimos anteriormente, para então calcular e criar os elementos da lista.

É esta criação sob demanda que chamamos de *lazy evaluation*, e é um dos fatores que permitem a economia de memória dos iteradores comentada acima.

A mesma coisa ocorre em funções como ```map()``` ou ```filter()```:

In [27]:
# aproveitemos a classe definida anteriormente Caminhada_iterador
lista_iteradores = [Caminhada_iterador(), Caminhada_iterador(3), Caminhada_iterador(10)]
m = map(next, lista_iteradores)
print(m)
print(list(m))

[0, 3, 10]


In [28]:
# uso errado da função map(), pois falta um argumento a pow():
m = map(pow, [1,2,3,4])
print(f'Objeto m criado com sucesso: {m}')
print('Ainda não retornou erro! Expressão não foi avaliada por preguiça.')
print('Agora sim avaliamos a expressão:')
list(m)

Objeto m criado com sucesso: <map object at 0x00000170FBF64790>
Ainda não retornou erro! Expressão não foi avaliada por preguiça.
Agora sim avaliamos a expressão:


TypeError: pow() missing required argument 'exp' (pos 2)

Toda vez que algum documento sobre alguma funcionalidade de Python te disser que expressões serão avaliadas preguiçosamente (*lazily evaluated*), pode saber que algum tipo de iteração está envolvida, implícita ou explicitamente.